# ***SQL Para Data Science - Prueba***.
### Nombre(s): Thomas Peet, Braulio Águila, Camilo Ramírez
### Generación: G47
### Profesores: José Terrazas - Sebastián Ulloa
### Fecha: 14-11-2022

## `Instrucciones: Para poder correr el notebook es necesario renombrar el archivo .env_example a .env y modificar su contenido agregando los datos de conexión del usuario en postgres`.

# Importación de librerías

In [ ]:
import os
import pandas as pd
import psycopg2
import psycopg2.extras as extras
import helpers
from dotenv import load_dotenv
import glob
import datetime

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from feature_engine.imputation import  MeanMedianImputer
from feature_engine.encoding import OrdinalEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

from sklearn import set_config
set_config(display='diagram')

---
# Parte 1

## Creación de la Base de Datos

In [ ]:
load_dotenv()
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_DBHOST')
dbname = os.getenv('POSTGRES_DBNAME')
drop_all = os.getenv('DROP_DATABASES')=='True'

if drop_all:
    conn = psycopg2.connect(user=user, host=host, port=5432, password=password, database='postgres')
    conn.set_session(autocommit=True)
    # Obtención de Cursor
    cursor = conn.cursor()
    # Eliminación de base de datos en caso de existir
    cursor.execute(f"DROP DATABASE IF EXISTS {dbname};")
    # Creacion de la base de datos en PostgreSQL
    cursor.execute(f"CREATE DATABASE {dbname};")

    conn.commit()
    cursor.close()
    conn.close()

## Obtención y tratamiento de las columnas del CSV

In [ ]:
df = pd.read_csv('train_cupid.csv')
df.rename(columns = {'hispanic / latin':'hispanic_latin'}, inplace = True)
df.columns = df.columns.str.replace(" ", "_")

cols = df.columns.to_list()
for index, col in enumerate(cols):
    cols[index] +=' numeric'
    
cols = ', '.join(cols)

## Creación de la tabla

In [ ]:
conn = psycopg2.connect(dbname=dbname, user=user, host=host, port=5432, password=password)

cursor = conn.cursor()

#Eliminando las tablas train_cupid y test_cupid si existen.
cursor.execute("DROP TABLE IF EXISTS train_cupid;")
cursor.execute("DROP TABLE IF EXISTS test_cupid;")

# Obteniendo un Cursor para las tablas
name_table_train = f"train_cupid ({cols})"
name_table_test = f"test_cupid ({cols})"

# Creación de las sentecias para las tablas
sqlTable_train = "create table "+name_table_train+';'
sqlTable_test = "create table "+name_table_test+';'

# Creando las tablas en PostgreSQL
cursor.execute(sqlTable_train)
cursor.execute(sqlTable_test)
conn.commit()

## Importación de los datos del CSV a las Tablas creadas

In [ ]:
with open('train_cupid.csv', 'r') as f:    
    next(f) # Saltar la fila de los encabezados del CSV.
    cursor.copy_from(f, 'train_cupid', sep=',')

with open('test_cupid.csv', 'r') as f:    
    next(f) # Saltar la fila de los encabezados del CSV.
    cursor.copy_from(f, 'test_cupid', sep=',')

conn.commit()

## Añadiendo columna indice a las tablas train_cupid y test_cupid

In [ ]:
cursor.execute('ALTER TABLE train_cupid ADD indice SERIAL PRIMARY KEY;')
cursor.execute('ALTER TABLE test_cupid ADD indice SERIAL PRIMARY KEY;')
conn.commit()
cursor.close()

---
# Parte 2

## Importando datos de train_cupid con Pandas

In [ ]:
df = pd.read_sql('SELECT * FROM train_cupid', conn, index_col='indice')
df.info()

## Segmentación de la data para los 3 vectores objetivos: [**single**, **seeing_someone**, **available**]

In [ ]:
targets = ['single', 'seeing_someone', 'available']
data = {}
for target in targets:
    X = df.drop(columns=[target])
    y = df[target]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.2, random_state=42)
    data[target] = {
        'X': X,
        'y': y,
        'X_train': X_train,
        'X_valid': X_valid,
        'y_train': y_train,
        'y_valid': y_valid,
    }

## Definición de los clasificadores a utilizar

In [ ]:
modelos = {
    'm1' : GradientBoostingClassifier(random_state=42),
    'm2' : AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=1, n_estimators=5), random_state=42, n_estimators=100, learning_rate=1),
    'm3' : RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10),
    'm4' : SVC(random_state=42, probability=True),
    'm5' : DecisionTreeClassifier(random_state=42),
    'm6' : LogisticRegression(random_state=42, C=0.01),
    'm7' : BernoulliNB()
}

## Preprocesamiento

In [ ]:
prep = Pipeline(steps=[
    ('num_imp', MeanMedianImputer(imputation_method='mean')),
    ('ord', OrdinalEncoder(encoding_method='ordered', variables='age', ignore_format=True)),
    ('sc', SklearnTransformerWrapper(StandardScaler(), variables=['age', 'height']))
])

## Fit y serialización de los modelos

In [ ]:
for target in targets:
    print(target)
    for modelo in modelos.values():
        model_f = {'prep':prep, 'classifier':modelo}
        helpers.report_performance(
            # Esta función genera un pipeline con el subpipeline de preprocess y el modelo a entrenar.
            helpers.pipeline_maker(**model_f),  
            str(modelo.__class__).replace("'>", '').split('.')[-1],
            target,
            data[target]['X_train'],
            data[target]['X_valid'],
            data[target]['y_train'],
            data[target]['y_valid']
        )

---
# Parte 3

## Importando datos de test_cupid con Pandas

In [ ]:
df_test = pd.read_sql('SELECT * FROM test_cupid', conn, index_col='indice')
df_test

## Función para insertar datos en una tabla

In [ ]:
def execute_values(conn, df, table):

    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    
    # SQL query que se ejecutará
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    print(f'\t\t{query}')
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("\t\tthe dataframe is inserted")
    cursor.close()

## Queries que se deben evaluar

In [ ]:
queries = {
    'Query 1' : ['atheism', 'asian', 'employed', 'pro_dogs', 'chinese'],
    'Query 2' : ['income_over_75', 'french', 'german', 'orientation_straight', 'new_york'],
    'Query 3' : ['education_undergrad_university', 'body_type_regular', 'pro_dogs', 'employed'],
    'Query 4' : ['taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism']
}

## Evaluación de los modelos y guardado de las tablas tabulación cruzada de las predicciones en la base de datos.

In [17]:
cursor_test = conn.cursor()
for target in targets:
    print(target)

    # Segmentación de la data para el test
    X_test = df_test.drop(columns=[target])
    y_test = df_test[target]
    
    # Fecha de los modelos que se utilzaran. Formato: 1211-19 = 12 de Noviembre a las 19 horas. Se utilizará por defecto la fecha actual
    fecha = datetime.datetime.now().strftime('%d%m-%H')
    
    # Listado de los modelos serializados
    archivos_de_modelos = glob.glob(f'./models/{target}*_{fecha}.pkl')
    
    # Se evaluaran las 4 queries en los 7 modelos entrenados para predecir las 3 variables objetivos. Es decir se evaluaran 84 modelos.
    for key, query in queries.items():
        print(f'\t{key}({query})')
        for archivo_modelo in archivos_de_modelos:
            print(f'\t\tModelo: {archivo_modelo}')

            # Se hace una predicción sobre la data de test y se crea una tabla de tabulación cruzada sobre la data usando las variables de la query
            queries_result_for_this_model, target_name, model_name = helpers.create_crosstab(archivo_modelo, X_test, y_test, query)
            
            # Se genera una tabla usando los datos del vector objetivo, nombre del modelo y query utilizada
            nombre_tabla = f"{target}_{model_name.lower()}_{key.lower().replace(' ','')}"

            # Las columnas que llevará esta tabla son las de la query más una variable y_hat que tendrá el promedio de la predicción para cada combinación
            cols = ' numeric ,'.join(list(queries_result_for_this_model.index.names))+' numeric'
            
            # Se elimina si existe y se crea la tabla
            cursor_test.execute(f"DROP TABLE IF EXISTS {nombre_tabla};")
            cursor_test.execute(f"CREATE TABLE {nombre_tabla} ({cols}, {target}_yhat numeric);")
            conn.commit()            

            # Se insertan los valores de la tabulación cruzada en la tabla
            execute_values(conn, queries_result_for_this_model.reset_index(), nombre_tabla)

cursor_test.close()
conn.close()

		INSERT INTO seeing_someone_svc_query4(taurus,indian,washington,income_between_50_75,hinduism,seeing_someone_yhat) VALUES %s
		the dataframe is inserted
available
	Query 1(['atheism', 'asian', 'employed', 'pro_dogs', 'chinese'])
		Modelo: ./models\available__AdaBoostClassifier__1411-20.pkl
		INSERT INTO available_adaboostclassifier_query1(atheism,asian,employed,pro_dogs,chinese,available_yhat) VALUES %s
		the dataframe is inserted
		Modelo: ./models\available__BernoulliNB__1411-20.pkl
		INSERT INTO available_bernoullinb_query1(atheism,asian,employed,pro_dogs,chinese,available_yhat) VALUES %s
		the dataframe is inserted
		Modelo: ./models\available__DecisionTreeClassifier__1411-20.pkl
		INSERT INTO available_decisiontreeclassifier_query1(atheism,asian,employed,pro_dogs,chinese,available_yhat) VALUES %s
		the dataframe is inserted
		Modelo: ./models\available__GradientBoostingClassifier__1411-20.pkl
		INSERT INTO available_gradientboostingclassifier_query1(atheism,asian,employed,pro_dog